<a href="https://colab.research.google.com/github/tanupunjani/Mastering-tensorflow/blob/main/Emotion_Detection_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install aicrowd-cli

     |████████████████████████████████| 43 kB 600 kB/s 
     |████████████████████████████████| 54 kB 1.6 MB/s 
     |████████████████████████████████| 62 kB 436 kB/s 
     |████████████████████████████████| 211 kB 30.5 MB/s 
     |████████████████████████████████| 170 kB 36.4 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 51 kB 4.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
API_KEY = '8db8021f6b44ed001637328f11a49992' # Please get your your API Key from [https://www.aicrowd.com/participants/me]
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


In [3]:
# Downloading the Dataset
!mkdir data
!aicrowd dataset download --challenge emotion-detection -j 3 -o data

test.csv: 100% 642k/642k [00:00<00:00, 982kB/s]
train.csv:   0% 0.00/2.30M [00:00<?, ?B/s]
val.csv:   0% 0.00/262k [00:00<?, ?B/s]
val.csv: 100% 262k/262k [00:00<00:00, 515kB/s]
train.csv: 100% 2.30M/2.30M [00:00<00:00, 2.43MB/s]


#Reading the data

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

train_data = pd.read_csv("/content/data/train.csv")
valid_data = pd.read_csv("/content/data/val.csv")
test_data = pd.read_csv("/content/data/test.csv")

In [5]:
train_data.head()

,text,label
0,takes no time to copy/paste a press release,0
1,You're delusional,1
2,Jazz fan here. I completely feel. Lindsay Mann...,0
3,ah i was also confused but i think they mean f...,0
4,Thank you so much. ♥️ that means a lot.,0


#Problem definition

##The problem aims to tell a computer to distinguish between positive and negative emotion. The model should be able to predict if its 1 (meaning positive) or 0 (meaning negative)

#Visualize the data

In [6]:
test_data

,text,label
0,I was already over the edge with Cassie Zamora...,0
1,I think you're right. She has oodles of cash a...,0
2,Haha I love this. I used to give mine phone bo...,1
3,Probably out of desperation as they going no a...,0
4,Sorry !! You’re real good at that!!,0
...,...,...
8677,Yeah no...I would find it very demeaning,1
8678,This is how mafia works,0
8679,Ah thanks 👍🏻,0
8680,I ask them straight why they don't respect my ...,1


In [7]:
#Shuffling training dataframes
train_df_shuffled = train_data.sample(frac=1, random_state=42)
train_df_shuffled.head()

,text,label
13976,Yaaaay!!,0
3256,says the insecure idiot who just had to dig th...,0
24501,"Game still ""fun"" if you don't tryhard and play...",0
22477,"Connie Drake learned to fight smarter, not har...",0
16658,"And she jumped into sewage. Yeh, no.",1


In [8]:
test_data.tail()

,text,label
8677,Yeah no...I would find it very demeaning,1
8678,This is how mafia works,0
8679,Ah thanks 👍🏻,0
8680,I ask them straight why they don't respect my ...,1
8681,Annette Acosta also tends to out vote Annette ...,0


In [9]:
valid_data.head()

,text,label
0,While I agree with my political views could be...,0
1,im still starving,1
2,*Hey just noticed..* it's your **2nd Cakeday**...,0
3,They just did. Check out the sticky post.,0
4,"I hope so too, she deserves it.",0


In [10]:
test_data

,text,label
0,I was already over the edge with Cassie Zamora...,0
1,I think you're right. She has oodles of cash a...,0
2,Haha I love this. I used to give mine phone bo...,1
3,Probably out of desperation as they going no a...,0
4,Sorry !! You’re real good at that!!,0
...,...,...
8677,Yeah no...I would find it very demeaning,1
8678,This is how mafia works,0
8679,Ah thanks 👍🏻,0
8680,I ask them straight why they don't respect my ...,1


In [11]:
#counting the total samples
len(train_data)

31255

In [12]:
len(test_data)

8682

In [13]:
len(valid_data)

3473

#Visualize the samples

##Lets visualize 5 random samples of train sentences at a time 

In [14]:
import random
random_index = random.randint(0, len(train_data)-5) #create indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "label"]][random_index:random_index+5].itertuples():
  _,text, label = row
  print(f"Label : {label}", "(Positive)") if label > 0 else "(negative)"
  print(f"Text:\n{text}\n")
  print("---\n")

Text:
Let people worship if they want to. It’s time for us to move forward and focus on bettering our own lives

---

Text:
shouldve just given him the sum you usually pay and transfer the burden of awkwardness onto him

---

Text:
Yeah, right. Jacob Jones can marry Jacob Jones for that matter, as it seems only his genius is keeping her on the throne.

---

Text:
Lauren Smith seems to be trying to get the foul over making the 3 a lot this season.

---

Text:
Most national subreddits are nowhere near *real* people, it's quite interesting to see the disparities.

---



In [15]:
#Convert abstract text lines into lists
train_sentences = train_df_shuffled["text"].to_numpy()
val_sentences = valid_data["text"].to_numpy()
test_sentences = test_data["text"].to_numpy()

In [16]:
train_sentences[:10]

array(['Yaaaay!!',
       'says the insecure idiot who just had to dig through my comment history to try to get back at me. lol creep',
       'Game still "fun" if you don\'t tryhard and play on QM but some people are trolling on other mode and that\'s Can be hard for nerv.',
       'Connie Drake learned to fight smarter, not harder. I love her development so much',
       'And she jumped into sewage. Yeh, no.',
       'It’s my favorite part of the whole week',
       'One of he most useful things I’ve read on this site as a medical student. Thank you very much.',
       "As a contractor this is a huge nono. If it's not on paper it not on your wall either.",
       'It scares me that left to right is also possible for anyone',
       'They really need to bring Amanda Weiss son back to do more flashback scenes of young Amanda Weiss. That would be SO GOOD'],
      dtype=object)

In [17]:
len(train_sentences)

31255

In [18]:
len(val_sentences)

3473

In [19]:
train_labels = train_df_shuffled["label"].to_numpy()
val_labels = valid_data["label"].to_numpy()
test_labels = test_data["label"].to_numpy()

In [20]:
train_labels[:10]

array([0, 0, 0, 0, 1, 0, 0, 1, 1, 0])

#Text vectorization 

In [21]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#Use the default text vectorization variables
text_vectorizer = TextVectorization(max_tokens=None, #how many words in the vocabulary (all different kinds of text)
                                    standardize = "lower_and_strip_punctuation",#how to process text
                                    split = "whitespace", #how to split the tokens
                                    ngrams = None, #create group of n-words
                                    output_mode = "int", #how to map tokens to numbers
                                    output_sequence_length=None)

In [22]:
#finding average numner of tokens in our reviews text
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

13

In [23]:
max_vocab_length = 1000 #max number of words to have in our vocabulary
max_lenght = 13 #max_length of our sequnces

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_sequence_length=13)

# Map text_vectorizer to train sentences

In [24]:
text_vectorizer.adapt(train_sentences)

In [25]:
#Creating a sample and tokenizing it
sample_sentence = "Tanvi is an expert AI Engineer"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 13), dtype=int64, numpy=array([[ 1,  8, 56,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0]])>

#Cool ! Now we can choose any random sentence from our train sentences to check the text vectorization feature

In [26]:
random_sentence = random.choice(train_sentences)
print(f"Orignal sentence:\n{random_sentence}\n\nVectorized sentence:")

text_vectorizer([random_sentence])

Orignal sentence:
She does have a great ass I’ve always thought so

Vectorized sentence:


<tf.Tensor: shape=(1, 13), dtype=int64, numpy=array([[ 59, 166,  23,   5, 106, 625, 329, 149, 148,  21,   0,   0,   0]])>

#Find the top 5 words and the bottom 5 words

In [27]:
words_in_vocab = text_vectorizer.get_vocabulary()

In [28]:
top_5_words = words_in_vocab[:5]

In [29]:
top_5_words

['', '[UNK]', 'the', 'i', 'to']

In [30]:
bottom_5_words = words_in_vocab[-5]
bottom_5_words

'bed'

#Creating an embedding layer

In [31]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length = max_lenght)
embedding

#Testing the embeddig layer on random sentences

In [32]:
random_sentence = random.choice(train_sentences)
random_embedding = embedding(text_vectorizer([random_sentence]))
print(f"Random sentence: {random_sentence}")
random_embedding

Random sentence: Ha ha manu plays shit football amirite guys


<tf.Tensor: shape=(1, 13, 128), dtype=float32, numpy=
array([[[-0.00249169,  0.00039764,  0.03355694, ..., -0.00961616,
          0.04543768, -0.0253271 ],
        [-0.00249169,  0.00039764,  0.03355694, ..., -0.00961616,
          0.04543768, -0.0253271 ],
        [-0.00309793,  0.00090615,  0.04355029, ...,  0.02279839,
         -0.00905225, -0.04904077],
        ...,
        [-0.02963591,  0.03810528, -0.02284383, ..., -0.04210591,
         -0.03589249, -0.01557285],
        [-0.02963591,  0.03810528, -0.02284383, ..., -0.04210591,
         -0.03589249, -0.01557285],
        [-0.02963591,  0.03810528, -0.02284383, ..., -0.04210591,
         -0.03589249, -0.01557285]]], dtype=float32)>

In [33]:
random_embedding[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.00249169,  0.00039764,  0.03355694,  0.01102369, -0.04004641,
        0.02127457,  0.03642469, -0.00845911,  0.04466   , -0.02467129,
       -0.04276622, -0.04650741,  0.0302625 ,  0.00501063, -0.02073274,
        0.04076665, -0.04440799, -0.01347589,  0.0450828 ,  0.01239821,
        0.00723379,  0.01288632, -0.01122186,  0.04280797, -0.00955522,
        0.04660921,  0.04811851, -0.03836304,  0.00790018,  0.00783   ,
        0.01260884,  0.02863816, -0.00507995, -0.03304418,  0.046605  ,
       -0.00229287,  0.02241473, -0.0487495 , -0.02474066,  0.01158885,
       -0.02434257,  0.04666995, -0.04156928, -0.02865679,  0.01220808,
        0.01406441,  0.01831507,  0.03389328,  0.01851174, -0.0023215 ,
        0.04882671, -0.04516926,  0.01063726,  0.01105471,  0.04860776,
       -0.01238495, -0.03673091, -0.01368611,  0.04675848, -0.02054814,
       -0.00179414, -0.03340197,  0.00495118,  0.02819309,  0.03200645,
        0.023082

#Getting the helper functions 

In [34]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-09-05 23:03:37--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-09-05 23:03:38 (79.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [35]:
from helper_functions import calculate_results, create_tensorboard_callback, accuracy_score ,precision_recall_fscore_support

#Modelling a text dataset

### We are creating a benchmark for our experiments

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#Creating tokenization and modelling pipeline

model_0 = Pipeline ([
                     ("tfidf", TfidfVectorizer()), #convert words to numbers using tfidf
                     ("clf", MultinomialNB()) #model the text
])

#Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

#Get the score

In [37]:
baseline_score = model_0.score(val_sentences,val_labels)

In [38]:
print(f"{baseline_score*100:.2f}%")

79.36%


In [39]:
#Make the predictions 
baseline_preds = model_0.predict(val_sentences)
baseline_preds

array([0, 0, 0, ..., 0, 0, 0])

#Using the transfer learning for our project

In [40]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


In [41]:
embed_samples = embed([sample_sentence,
                       "When you call the universal sentence encoder on a sentence, it turns it into numbers."])
print(embed_samples[0][:50])

tf.Tensor(
[ 0.06579109  0.01226765 -0.03711271 -0.00360206  0.05657576 -0.03709659
 -0.0402483  -0.04691624  0.04751117 -0.00626177  0.03339733 -0.03225748
 -0.02951182  0.05706123 -0.03125783 -0.01789836 -0.02796606  0.01856283
 -0.0374579  -0.0888266  -0.01428857  0.01822379 -0.00720385  0.01422425
  0.01323723 -0.00044181 -0.00970582 -0.00105472 -0.04080107 -0.02575576
 -0.04882491  0.03869063 -0.01404163  0.05813107 -0.03477886  0.06209134
 -0.08300394 -0.06032129 -0.0708599   0.05222806  0.03551109 -0.05332186
  0.00364079  0.04148    -0.05663498  0.07072239  0.01188438 -0.03316613
  0.0146693  -0.02381066], shape=(50,), dtype=float32)


In [42]:
embed_samples.shape

TensorShape([2, 512])

In [43]:
#Convert USE layer to keras layer
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], #shape of inputs coming
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [44]:
#Create the model
model_1 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64,activation="relu"),
    layers.Dense(1, activation="sigmoid")
],name="model_1_USE")

In [45]:
#compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


In [46]:
model_1.summary()

Model: "model_1_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
dense (Dense)                (None, 64)                32832     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [47]:
SAVE_DIR = "model_logs"

In [48]:
from helper_functions import create_tensorboard_callback

In [49]:
#Fit the data
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data = (val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="tf_hub_sentence_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20210905-230411
Epoch 1/5
977/977 [==============================] - 21s 19ms/step - loss: 0.4169 - accuracy: 0.8122 - val_loss: 0.3822 - val_accuracy: 0.8200
Epoch 2/5
977/977 [==============================] - 17s 17ms/step - loss: 0.3818 - accuracy: 0.8253 - val_loss: 0.3818 - val_accuracy: 0.8246
Epoch 3/5
977/977 [==============================] - 17s 17ms/step - loss: 0.3723 - accuracy: 0.8307 - val_loss: 0.3761 - val_accuracy: 0.8223
Epoch 4/5
977/977 [==============================] - 18s 19ms/step - loss: 0.3606 - accuracy: 0.8378 - val_loss: 0.3741 - val_accuracy: 0.8218
Epoch 5/5
977/977 [==============================] - 17s 17ms/step - loss: 0.3495 - accuracy: 0.8434 - val_loss: 0.3741 - val_accuracy: 0.8238


In [50]:
model_1_preds = model_1.predict(val_sentences)
model_1_preds

array([[0.383918  ],
       [0.64990586],
       [0.00597529],
       ...,
       [0.04681696],
       [0.14647442],
       [0.46213815]], dtype=float32)

In [51]:
model_1_preds_probs = tf.round(model_1_preds)
model_1_preds_probs

<tf.Tensor: shape=(3473, 1), dtype=float32, numpy=
array([[0.],
       [1.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)>

In [52]:
#Calculating results
model_1_results = calculate_results(val_labels, model_1_preds_probs)
model_1_results

{'accuracy': 82.37834725021595,
 'f1': 0.8082435564305078,
 'precision': 0.8062120981517306,
 'recall': 0.8237834725021596}

In [53]:
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred = baseline_preds)
baseline_results

{'accuracy': 79.3550244745177,
 'f1': 0.7052936108398501,
 'precision': 0.8174126292071061,
 'recall': 0.7935502447451771}

#Comparing it to our baseline

#Create a helper function to compare our baseline results to new model results

In [54]:
def compare_baseline_to_new_results(baseline_results, new_model_results):
  for key, value in baseline_results.items():
    print(f"Baseline {key}:{value:.2f}, New{key}: {new_model_results[key]:.2f}, Difference:{new_model_results[key]-value:.2f}")

    

In [55]:
compare_baseline_to_new_results(baseline_results=baseline_results,
                                    new_model_results=model_1_results)

Baseline accuracy:79.36, Newaccuracy: 82.38, Difference:3.02
Baseline precision:0.82, Newprecision: 0.81, Difference:-0.01
Baseline recall:0.79, Newrecall: 0.82, Difference:0.03
Baseline f1:0.71, Newf1: 0.81, Difference:0.10


#Fine tuning our USE model

In [56]:
#Convert USE layer to keras layer
sentence_encoder_layer_fine_tuning = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], #shape of inputs coming
                                        dtype=tf.string,
                                        trainable=True,
                                        name="USE")

In [57]:
#Create the model
model_2 = tf.keras.Sequential([
    sentence_encoder_layer_fine_tuning,
    layers.Dense(128,activation="relu"),
    layers.Dense(1, activation="sigmoid")
],name="model_1_USE")


In [58]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [59]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=3,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                      "tf_hub_USE_fine_tuning")])

Saving TensorBoard log files to: model_logs/tf_hub_USE_fine_tuning/20210905-230642
Epoch 1/3
977/977 [==============================] - 235s 234ms/step - loss: 0.3887 - accuracy: 0.8268 - val_loss: 0.3544 - val_accuracy: 0.8425
Epoch 2/3
977/977 [==============================] - 228s 233ms/step - loss: 0.1537 - accuracy: 0.9400 - val_loss: 0.4232 - val_accuracy: 0.8362
Epoch 3/3
977/977 [==============================] - 227s 233ms/step - loss: 0.0263 - accuracy: 0.9914 - val_loss: 0.8155 - val_accuracy: 0.7964


In [60]:
model_2_preds = model_2.predict(val_sentences)
model_2_preds

array([[3.6801223e-03],
       [9.5350480e-01],
       [1.3202550e-04],
       ...,
       [2.7959800e-04],
       [6.1902869e-04],
       [9.9834776e-01]], dtype=float32)

In [61]:
model_2_preds_probs = tf.round(model_2_preds)
model_2_preds_probs

<tf.Tensor: shape=(3473, 1), dtype=float32, numpy=
array([[0.],
       [1.],
       [0.],
       ...,
       [0.],
       [0.],
       [1.]], dtype=float32)>

In [62]:
model_2_results = calculate_results(val_labels,
                  model_2_preds_probs)
model_2_results

{'accuracy': 79.64295997696516,
 'f1': 0.804665753122935,
 'precision': 0.8180489456515274,
 'recall': 0.7964295997696516}

In [63]:
compare_baseline_to_new_results(baseline_results, model_2_results)

Baseline accuracy:79.36, Newaccuracy: 79.64, Difference:0.29
Baseline precision:0.82, Newprecision: 0.82, Difference:0.00
Baseline recall:0.79, Newrecall: 0.80, Difference:0.00
Baseline f1:0.71, Newf1: 0.80, Difference:0.10


In [64]:
model_2_test_pred = model_2.predict(test_sentences)
model_2_test_pred

array([[9.9814665e-01],
       [5.2059339e-03],
       [1.5099284e-04],
       ...,
       [3.8209918e-04],
       [8.2416832e-01],
       [3.0873576e-04]], dtype=float32)

In [65]:
model_2_test_preds_probs = tf.round(model_2_test_pred)
model_2_test_preds_probs

<tf.Tensor: shape=(8682, 1), dtype=float32, numpy=
array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [1.],
       [0.]], dtype=float32)>

In [66]:
model_2_test_results = calculate_results(test_labels,model_2_test_preds_probs)
model_2_test_results

{'accuracy': 49.78115641557245,
 'f1': 0.4720959540344993,
 'precision': 0.49932914976757015,
 'recall': 0.4978115641557245}

In [67]:
compare_baseline_to_new_results(baseline_results, model_2_test_results)

Baseline accuracy:79.36, Newaccuracy: 49.78, Difference:-29.57
Baseline precision:0.82, Newprecision: 0.50, Difference:-0.32
Baseline recall:0.79, Newrecall: 0.50, Difference:-0.30
Baseline f1:0.71, Newf1: 0.47, Difference:-0.23


In [68]:
!pip install streamlit

     |████████████████████████████████| 8.0 MB 4.1 MB/s 
     |████████████████████████████████| 4.3 MB 39.8 MB/s 
     |████████████████████████████████| 75 kB 4.4 MB/s 
     |████████████████████████████████| 111 kB 49.1 MB/s 
     |████████████████████████████████| 123 kB 48.4 MB/s 
     |████████████████████████████████| 787 kB 30.8 MB/s 
     |████████████████████████████████| 370 kB 37.8 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=baddebeec39add04d47f178ce5dddd47f77c3f659f753502bd2662e8a29b1571
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfu

In [69]:
!streamlit hello


  Welcome to Streamlit. Check out our demo in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.194.229.143:8501

  Ready to create your own Python apps super quickly?
  Head over to https://docs.streamlit.io

  May you create awesome apps!


  Stopping...
